In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PowerTransformer


In [2]:
df = pd.read_csv('5000_old_car.csv')

In [3]:
df

,Unnamed: 0,id,url,location,car_body_type,price,make,model,variant,year,petrol,transmission,mileage,number_of_owners
0,0,1699779940,"Ford Endeavour 3.2 Titanium AT 4X4, 2019, Diesel",Delhi,SUV,3150000.0,ford,ford-endeavour,version-Endeavour 3.2 Titanium AT 4x4-3417,2019.0,diesel,1.0,70000.0,1.0
1,1,1699442686,"Maruti Suzuki Zen Estilo LXI BSIII, 2008, CNG ...",Gujarat,HATCHBACK,125000.0,maruti-suzuki,maruti-suzuki-zen-estilo,zen-estilo-lxi-bsiii,2008.0,cng,2.0,106402.0,2.0
2,2,1698168319,"Honda City 2008-2011 1.5 S MT, 2009, Petrol",Delhi,SEDAN,199000.0,cars-honda,cars-honda-city,version-City 1.5 S MT-757,2009.0,petrol,2.0,79000.0,2.0
3,3,1698168128,"Maruti Suzuki Swift Dzire VDI, 2016, Diesel",West Bengal,SEDAN,475000.0,maruti-suzuki,maruti-suzuki-swift-dzire,version-Swift Dzire VDI-1564,2016.0,diesel,2.0,53000.0,1.0
4,4,1698168099,"Mercedes-Benz C Class Prime 200, 2011, Petrol",Delhi,OTHERS,850000.0,mercedes-benz,mercedes-benz-c-class-2,mercedes-benz-c-class-2-prime-c-200,2011.0,petrol,1.0,67800.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002,5,1700139404,"Hyundai Fluidic Verna 1.6 CRDi S, 2012, Diesel",Telangana,SEDAN,450000.0,hyundai,hyundai-fluidic-verna,version-Fluidic Verna 1.6 CRDi S-898,2012.0,diesel,2.0,102272.0,1.0
6003,6,1666728972,Skoda Fabia 2011 Diesel Well Maintained,Tamil Nadu,SEDAN,159999.0,skoda,skoda-fabia,version-Fabia Elegance 1.2 TDI-2017,2011.0,diesel,2.0,130000.0,3.0
6004,7,1697252431,"Mahindra Bolero, 2021, Diesel",Karnataka,SUV,1225000.0,mahindra,mahindra-bolero,mahindra-bolero-b6-(o),2021.0,diesel,2.0,11590.0,1.0
6005,8,1698090845,"Hyundai EON Era +, 2014, Petrol",Kerala,HATCHBACK,255000.0,hyundai,hyundai-eon,version-Eon Era +-889,2014.0,petrol,2.0,65000.0,2.0


In [4]:
df.shape

(6007, 14)

In [5]:
df.dtypes

Unnamed: 0            int64
id                    int64
url                  object
location             object
car_body_type        object
price               float64
make                 object
model                object
variant              object
year                float64
petrol               object
transmission        float64
mileage             float64
number_of_owners    float64
dtype: object

In [6]:
df.isnull().sum()

Unnamed: 0            0
id                    0
url                   0
location              0
car_body_type         0
price                 0
make                  0
model                 0
variant             176
year                  0
petrol               85
transmission        167
mileage              21
number_of_owners    194
dtype: int64

In [7]:
df.columns

Index(['Unnamed: 0', 'id', 'url', 'location', 'car_body_type', 'price', 'make',
       'model', 'variant', 'year', 'petrol', 'transmission', 'mileage',
       'number_of_owners'],
      dtype='object')

In [8]:
# Droping Id index and url
df.drop(['Unnamed: 0', 'id', 'url' ],axis=1,inplace=True)

In [9]:
df.head()

,location,car_body_type,price,make,model,variant,year,petrol,transmission,mileage,number_of_owners
0,Delhi,SUV,3150000.0,ford,ford-endeavour,version-Endeavour 3.2 Titanium AT 4x4-3417,2019.0,diesel,1.0,70000.0,1.0
1,Gujarat,HATCHBACK,125000.0,maruti-suzuki,maruti-suzuki-zen-estilo,zen-estilo-lxi-bsiii,2008.0,cng,2.0,106402.0,2.0
2,Delhi,SEDAN,199000.0,cars-honda,cars-honda-city,version-City 1.5 S MT-757,2009.0,petrol,2.0,79000.0,2.0
3,West Bengal,SEDAN,475000.0,maruti-suzuki,maruti-suzuki-swift-dzire,version-Swift Dzire VDI-1564,2016.0,diesel,2.0,53000.0,1.0
4,Delhi,OTHERS,850000.0,mercedes-benz,mercedes-benz-c-class-2,mercedes-benz-c-class-2-prime-c-200,2011.0,petrol,1.0,67800.0,2.0


In [10]:
# Replace Null Values with NOt avaliable

for i,column_data in enumerate(df.dtypes.items()):
    column, dtype = column_data
    if dtype == 'object':
        df[column].fillna('Not avaliable',inplace=True)
    else:
        df[column].fillna(method='pad',inplace=True)

In [11]:
df.isna().sum()

location            0
car_body_type       0
price               0
make                0
model               0
variant             0
year                0
petrol              0
transmission        0
mileage             0
number_of_owners    0
dtype: int64

In [12]:
df[['transmission', 'number_of_owners']] = df[['transmission', 'number_of_owners']].astype('int').astype('object')

In [13]:
df.head()

,location,car_body_type,price,make,model,variant,year,petrol,transmission,mileage,number_of_owners
0,Delhi,SUV,3150000.0,ford,ford-endeavour,version-Endeavour 3.2 Titanium AT 4x4-3417,2019.0,diesel,1,70000.0,1
1,Gujarat,HATCHBACK,125000.0,maruti-suzuki,maruti-suzuki-zen-estilo,zen-estilo-lxi-bsiii,2008.0,cng,2,106402.0,2
2,Delhi,SEDAN,199000.0,cars-honda,cars-honda-city,version-City 1.5 S MT-757,2009.0,petrol,2,79000.0,2
3,West Bengal,SEDAN,475000.0,maruti-suzuki,maruti-suzuki-swift-dzire,version-Swift Dzire VDI-1564,2016.0,diesel,2,53000.0,1
4,Delhi,OTHERS,850000.0,mercedes-benz,mercedes-benz-c-class-2,mercedes-benz-c-class-2-prime-c-200,2011.0,petrol,1,67800.0,2


In [15]:
sns.heatmap(df.corr(), cmap='YlGnBu_r')

ValueError: 'reds' is not a valid value for name; supported values are 'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r

# Univariate Analysis

In [ ]:
plt.figure(figsize=(15,100))
for i,column_data in enumerate(df.dtypes.items()):
    plt.subplot(25,1,i+1)
    plt.subplots_adjust(hspace=2)
    column, dtype = column_data
    if dtype == 'object':
        plt.xticks(rotation=90)
        sns.histplot(df[column])
    else:
        plt.xticks(rotation=90)
        sns.histplot(df[column],kde=True)

In [ ]:
#Bivariate analysis 
plt.figure(figsize=[15,200])
j = 0 
for column, dtype in df.drop('price',axis=1).dtypes.items():
    if dtype == 'object':
        plt.subplot(30,1,j+1)  
        plt.xticks(rotation=90)
        plt.subplots_adjust(hspace=1)
        plt.title(column + " vs Price", fontsize = 20)
        sns.barplot(x = df[column],y = df['price'],)
        j += 1

# Label Encoding

In [ ]:
obj_columns = df.select_dtypes('object').columns

In [ ]:
df[obj_columns] = df[obj_columns].apply(LabelEncoder().fit_transform)

In [ ]:
df

# Checking for outliers 

In [ ]:
df.boxplot(figsize=(15,15))

In [ ]:
X = df.drop('price',axis=1)
y = df['price']

In [ ]:
X_transfomred = MinMaxScaler().fit_transform(X)
X = pd.DataFrame(X_transfomred,columns=X.columns)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=True,shuffle=True,test_size=.33)

In [ ]:
models = [LinearRegression(),RandomForestRegressor(),DecisionTreeRegressor(),KNeighborsRegressor(),GradientBoostingRegressor(),Ridge(),SVR()]

In [ ]:
for model in models:
    print('model ',model)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    r2_s = r2_score(y_test,y_pred)
    print('R2 score:',r2_s)
    print('Mean Absoulte Error: ',mean_absolute_error(y_test,y_pred))
    print('Mean Sqaure Error: ',mean_squared_error(y_test,y_pred))
    print('Root Mean Sqaure:',r2_score(y_test,y_pred))
    score = cross_val_score(model,X,y,cv=10)
    print('Score:',score)
    print('cross val score mean: ',score.mean())
    print('diffrence between r2 score - cross val score: ',r2_s-score.mean())
    print('-'*50)

# Hyper parameter tunnign

In [ ]:
param_grid = {
    'max_depth' : range(10,20),
    'criterion' : ['mse'],
    'max_features' : ['auto', 'sqrt'],
    'min_samples_leaf' : range(2,6)
}
gridSearchCV = GridSearchCV(RandomForestRegressor(),param_grid=param_grid,refit=True,verbose=3)

In [ ]:
gridSearchCV.fit(X_train,y_train)

In [ ]:
gridSearchCV.best_params_

In [ ]:
y_pred = gridSearchCV.best_estimator_.predict(X_test)

In [ ]:
r2_score(y_test,y_pred)

# Saving model

In [ ]:
joblib.dump(gridSearchCV.best_estimator_,'gridSearchCV.obj')